## Lectura y limpieza de Datos Llamadas 123

* cargar y limpiar cada df de los meses 
* Modificar en la tabla de string a date/time 
* Por tabla (de cada mes), puedes hacer un reporte con los datos limpios
* Hacer todo esto en todas las tablas de 123 Y generar la tabla maestra
* hacer reporte de tabal maestra

In [2]:
import os
import sys
import logging  
import pandas as pd

import re # libreria para administracion de expresiones regulares

from pandas_profiling import ProfileReport #para generar reporte


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))
from data_manipulator import get_input_data

bucket = 'esp_bigdata_jemm'
directory = 'BigData'

%load_ext autoreload
%autoreload 2

## Visualizando el diccionario de datos

In [3]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


Para estandarizar los campos de los dataframes, relizare una copia de las variables declaradas en el ***diccionario de datos*** en un arreglo

In [19]:
columName=[x for x in df_metadata['NOMBRE']]

Usando la libreria ***re*** y con un recorrido por la lista creada, se realiza un cambio del caracter ***' '*** a ***'_'***, es de anotar que para estas variables no habia una necesidad de quitar los espacios alrededor del strig con el método *strip()*

In [20]:
for index in range(len(columName)):
    columName[index]=re.sub('\s+', '_',columName[index])
columName

['FECHA_INCIDENTE',
 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
 'CODIGO_LOCALIDAD',
 'LOCALIDAD',
 'EDAD',
 'UNIDAD',
 'GENERO',
 'RED',
 'TIPO_INCIDENTE',
 'PRIORIDAD',
 'MES',
 'CLASIFICACION_FINAL']

## Ahora pasaremos a los datos crudos:

Inicialemte inicializaré un arreglo con los nombres de los archivos que tengo en el bucket.

In [21]:
dataRaw=['datos-abiertos-julio-2019.csv',
         'datos-abiertos-agosto-2019.csv',
         'datos-abiertos-septiembre-2019.csv',
         'datos-abiertos-noviembre_2019.csv',
         'datos_abiertos_cons_diciembre_2019.csv',
         'datos-abiertos-enero_2020.csv',
         'llamadas-de-urgencias-y-emergencias.csv'
     ]

defino un arreglo de dataframes

In [22]:
dfArrayCalls=list()
for i in range(len(dataRaw)):
    df= get_input_data(bucket=bucket,
                          initial_directory=directory,
                          filename = dataRaw[i])
    dfArrayCalls.append(df)

Ahora definire una funcion que renombre los nombres de las columnas de acuerdo a nuestro arreglo tomado de ***Diccionario de datos***

In [30]:
# renombrar los titulos de las columnas de df
def RenombrarColumns(index):
    dfArrayCalls[index].rename(
        columns = {
            dfArrayCalls[index].keys()[0]           : columName[0],
            dfArrayCalls[index].keys()[1]           : columName[1],
            dfArrayCalls[index].keys()[2]           : columName[2],
            dfArrayCalls[index].keys()[3]           : columName[3],
            dfArrayCalls[index].keys()[4]           : columName[4],
            dfArrayCalls[index].keys()[5]           : columName[5],
            dfArrayCalls[index].keys()[6]           : columName[6],
            dfArrayCalls[index].keys()[7]           : columName[7],
            dfArrayCalls[index].keys()[8]           : columName[8],
            dfArrayCalls[index].keys()[9]           : columName[9],
            dfArrayCalls[index].keys()[10]          : columName[10],
            dfArrayCalls[index].keys()[11]          : columName[11]
        },
        inplace=True #esto manipula en memoria cargada para muestra, no el datos en bucket
    )
#dfArrayCalls[0]

In [39]:
for i in range(len(dfArrayCalls)):
    print(f'cosaaaad{i}\n',dfArrayCalls[i].head())

cosaaaad0
        FECHA_INCIDENTE FECHA_INICIO_DESPLAZAMIENTO_MOVIL  CODIGO_LOCALIDAD  \
0  2019-07-01 00:05:08               2019-07-01 00:11:02                 8   
1  2019-07-01 00:06:57               2019-07-01 00:12:56                19   
2  2019-07-01 00:09:26               2019-07-01 00:10:26                 6   
3  2019-07-01 00:14:42               2019-07-01 00:15:47                 8   
4  2019-07-01 00:17:13               2019-07-01 00:31:13                10   

         LOCALIDAD      EDAD    UNIDAD     GENERO    RED  \
0          Kennedy        39      Años   FEMENINO  Norte   
1   Ciudad Bolivar        69      Años  MASCULINO    Sur   
2       Tunjuelito        25      Años  MASCULINO    Sur   
3          Kennedy        65      Años  MASCULINO    Sur   
4         Engativa  SIN_DATO  SIN_DATO   SIN_DATO  Norte   

                           TIPO_INCIDENTE PRIORIDAD    MES CLASIFICACION_FINAL  
0                 Dificultad Respiratoria      ALTA  JULIO            Traslado

In [33]:
for ind in range(len(columName)): 
    if dfArrayCalls[0].keys()[ind]!=columName[ind]:
        dfArrayCalls[0].keys().replace(to_replace=dfArrayCalls[0].keys()[ind], value=columName[ind], regex=True)
print(dfArrayCalls[0].keys())

AttributeError: 'Index' object has no attribute 'replace'

In [5]:
raw_data_agosto2019 = get_input_data(bucket=bucket,
                          initial_directory=directory,
                          filename = 'llamadas-de-urgencias-y-emergencias.csv')
print(raw_data_agosto2019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127314 entries, 0 to 127313
Data columns (total 12 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   FECHA_INCIDENTE                    127314 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  127314 non-null  object
 2   CODIGO LOCALIDAD                   127314 non-null  int64 
 3   LOCALIDAD                          127314 non-null  object
 4   EDAD                               127314 non-null  object
 5   UNIDAD                             127314 non-null  object
 6   GENERO                             127314 non-null  object
 7   RED                                127314 non-null  object
 8   TIPO_INCIDENTE                     127314 non-null  object
 9   PRIORIDAD                          127286 non-null  object
 10  MES                                127314 non-null  object
 11  CLASIFICACION FINAL                127314 non-null  

In [7]:
raw_data_agosto2019['FECHA_INICIO_DESPLAZAMIENTO_MOVIL']

0         2019-01 01 00:04:37
1         2019-01 01 00:08:06
2         2019-01 01 00:07:41
3         2019-01 01 00:07:46
4         2019-01 01 00:10:40
                 ...         
127309       30/06/2019 23:42
127310       30/06/2019 23:55
127311       30/06/2019 23:55
127312         1/07/2019 0:30
127313         1/07/2019 0:04
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 127314, dtype: object

In [ ]:
# renombrar los titulos de las columnas de df
raw_data_agosto2019.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_DE_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL',
        'TIPO DE INCIDIDENT'                : 'TIPO_DE_INCIDIDENTE'
    },
    inplace=True #esto manipula en memoria cargada para muestra, no el datos en bucket
)
raw_data_agosto2019.head()

In [ ]:
raw_data_agosto2019['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

In [ ]:
# de los valores de la columna LOCALIDAD a cada string se le limpia los espacios alrededor
raw_data_agosto2019['LOCALIDAD'] = raw_data_agosto2019['LOCALIDAD'].apply(lambda x: x.strip())

In [ ]:
#
raw_data_agosto2019['GENERO'] = raw_data['GENERO'].apply(lambda x: x.strip())
raw_data_agosto2019['GENERO'].value_counts(dropna=False) 

In [ ]:
# limpia y reemplaza en df los valores de la columna 'TIPO_INCIDENTE'
raw_data_agosto2019['TIPO_INCIDENTE'] = raw_data['TIPO_INCIDENTE'].apply(lambda x: x.strip())

In [ ]:
raw_data_agosto2019['CLASIFICACION_FINAL'].fillna('SIN_DATO', inplace=True)
raw_data_agosto2019['CLASIFICACION_FINAL'] = raw_data_agosto2019['CLASIFICACION_FINAL'].apply(lambda x: x.strip())
raw_data_agosto2019.head()

In [ ]:
raw_data_agosto2019['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].value_counts()

In [ ]:
date_time_str = raw_data_agosto2019['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'][5]
date_time_str

In [ ]:
date_time_str=='1/08/2019 0:10'

In [ ]:
from datetime import *
import numpy as np

In [ ]:
#"21/11/06 16:30" #"%d/%m/%y %H:%M"
date_time_obj = datetime.strptime('21/11/06 16:30','%d/%m/%y %H:%M')
date_time_obj

# datetime.datetime.strptime(input, format)

In [ ]:
date_time_str = '18/09/19 01:55:19'

date_time_obj = datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')
date_time_obj

In [ ]:
from datetime import datetime

In [ ]:
date='01/08/2019 12:00:00'
date_time=pd.to_datetime(date, format='%d/%m/%Y %H:%M:%S')
print (date_time, type (date_time))

In [ ]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
raw_data_agosto2019[col] = pd.to_datetime(raw_data_agosto2019[col], format='%d/%m/%Y %H:%M') 
raw_data_agosto2019[col].value_counts
# raw_data_ago.head()

In [ ]:
raw_data_agosto2019.head()

In [ ]:
raw_data_agosto2019['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].apply(
    for x in raw_data_agosto2019['FECHA_INICIO_DESPLAZAMIENTO_MOVIL']:x =datetime.strptime(x, '%d/%m/%y %H:%M:%S')       
    ).value_counts()

# generar Informe

In [ ]:
profile = ProfileReport(df=raw_data_agosto2019)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_agosto_2019.html')

In [ ]:
raw_data = get_input_data(bucket=bucket,
                          initial_directory=directory,
                          filename = 'datos-abiertos-julio-2019.csv')
print(raw_data.info())

It seems that this file contains already the fields, but need to rename them

### Rename Columns

In [ ]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True #esto manipula en memoria cargada para muestra, no el datos en bucket
)
raw_data.head()

### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [ ]:
raw_data['LOCALIDAD'].value_counts(dropna=False) #dropna no muestra los nulos en True por defecto

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [ ]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

In [ ]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [ ]:
raw_data.head()

In [ ]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

In [ ]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'CLASIFICACION_FINAL'
#raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [ ]:
raw_data[col].fillna('SIN_DATO').value_counts()

In [ ]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [ ]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_julio_2019.html')

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [ ]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

### Save the final table
use to_csv to save the table

In [ ]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_julio_2019.csv', encoding='latin1', sep=';', index=False)

In [ ]:
raw_data

In [ ]:
raw_data_agosto2019

In [ ]:
data = pd.concat([raw_data,raw_data_agosto2019])
data

In [ ]:
pd.concat?